## Tratamiento de valores atípicos


Un valor atípico o valor extremos (outlier) es un valor el cual es significativamente diferente del resto de los datos.“Un outlier es una observación la cual se desvía tanto del resto de las observaciones que levanta sospechas sobre el mecanismo que lo generó” [D. Hawkins. Identification of Outliers, Chapman and Hall , 1980].

Valores estadísticos como la media y la varianza son susceptibles a los valores extremos. Además, **algunos modelos de Machine Learning son sensitivos a los outliers** lo cual decrece su desempeño. Por lo tanto, dependiendo de cuál algoritmo deseas usar para entrenar el modelo, frecuentemente es necesario remover los valores atípicos de las variables.

Discutimos en la sección 3, cómo identificar los outliers. En esta sección vamos a discutir cómo podemos procesarlos para entrenar nuestros modelos de machine learning o aprendizaje de máquina.

Es importante resaltar que con cada modificación que hacemos en nuestros datos, introducimos algún sesgo. Por eso es tan importante saber cuáles son las implicaciones de cada método. Si es una buena decisión o no dependerá de la naturaleza de los datos que estamos analizando.

## Cómo podemos pre-procesar los valores extremos?

- Removerlos: eliminar los valores extremos de nuestro conjunto de datos
- Tratar los outliers como datos faltantes y proceder con cualquiera de las técnicas de sustitución
- Discretización: los datos son discretizados ( ver sección 8) y los valores atípicos son colados en los segmentos extremos junto con los valores más bajos y altos del conjunto de datos
- Truncamiento de valores: Limitar la distribución de la variable a unos valores máximos y mínimos. También se le conoce como codificación Top / Bottom 


**El truncamiento de valores** se conoce en inglés como capping, trimming, censoring o winsorization.


## Truncamiento de outliers.

**Truncar**, significa limitar los valores máximos y/o mínimos de una distribución a un valor arbitrario. En otras palabras, los valores más grandes o más pequeños que los que arbitrariamente se han determinado, son truncados.

Truncar puede hacerse en ambos extremos de la distribución, o solo en un extremo, dependiendo de la variable y el caso de uso.

Puedes ver la charla de Soledad en pydata Londres [pydata](https://www.youtube.com/watch?v=KHGGlozsRtA), donde ella presenta un ejemplo de truncamiento de los valores extremos en una compañía financiera.

Los números en los cuales se debe truncar la distribución pueden ser determinados: 

- arbitrariamente
- usando la regla de proximidad del rango inter-cuantil 
- usando la aproximación gaussiana 
- usando los cuantiles

### Ventajas

- no remueve las observaciones

### Limitaciones

- distorsiona la distribución de las variables 
- distorsiona la relación entre las variable

## En este Demo

Vas a aprender como truncar los valores extremos en las variables utilizando valores arbitrarios y el conjunto de datos del Titanic

## Importante

Cuando truncamos nuestros datos, tendemos a limitar los valores en el set de entrenamiento y en el set de prueba. Es importante recordar que los valores de truncamiento DEBEN SER derivados del set de entrenamiento. Y luego esos mismos valores se usan para truncar las variables en el set de prueba.

Para simplificar el demo, no lo haremos, pero por favor, ten eso en cuenta cuando construyas tus pipelines de machine learning.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from feature_engine import missing_data_imputers  as msi
from feature_engine import outlier_removers as outr

In [2]:
# función para cargar los datos del titanic 

def load_titanic():
    data = pd.read_csv('../titanic.csv')
    data['cabin'] = data['cabin'].astype(str).str[0]
    data['pclass'] = data['pclass'].astype('O')
    data['embarked'].fillna('C', inplace=True)
    return data

In [3]:
data = load_titanic()
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


## Truncador de outliers con valores arbitrarios con feature-engine

Los límites para truncar los valores extremos son determinados por el usuario. 

### Truncando el extremo superior

In [4]:
# encontremos cual el es valor máximo de la variable Age y 
# Fare en los datos del titanic

data.age.max(), data.fare.max()

(80.0, 512.3292)

In [5]:
# inicialicemos el ArbitraryOutlierCapper de feature-engine
capper = outr.ArbitraryOutlierCapper(max_capping_dict = {'age':50, 'fare':200},
                                     min_capping_dict = None)
capper.fit(data)

ArbitraryOutlierCapper(max_capping_dict=None, min_capping_dict=None)

In [6]:
capper.right_tail_caps_

{'age': 50, 'fare': 200}

In [7]:
capper.left_tail_caps_

{}

In [8]:
temp = capper.transform(data)

temp.age.max(), temp.fare.max()

(50.0, 200.0)

### Truncando el extremo inferior

In [9]:
capper = outr.ArbitraryOutlierCapper(max_capping_dict=None,
                                     min_capping_dict={
                                         'age': 10,
                                         'fare': 100
                                     })
capper.fit(data)

ArbitraryOutlierCapper(max_capping_dict=None, min_capping_dict=None)

In [10]:
capper.variables

['age', 'fare']

In [11]:
capper.right_tail_caps_

{}

In [12]:
capper.left_tail_caps_

{'age': 10, 'fare': 100}

In [13]:
temp = capper.transform(data)

temp.age.min(), temp.fare.min()

(10.0, 100.0)

###  Truncando ambos extremos 

In [14]:
capper = outr.ArbitraryOutlierCapper(max_capping_dict={
                                     'age': 50, 'fare': 200},
                                     min_capping_dict={
                                     'age': 10, 'fare': 100})
capper.fit(data)

ArbitraryOutlierCapper(max_capping_dict=None, min_capping_dict=None)

In [15]:
capper.right_tail_caps_

{'age': 50, 'fare': 200}

In [16]:
capper.left_tail_caps_

{'age': 10, 'fare': 100}

In [17]:
temp = capper.transform(data)

temp.age.min(), temp.fare.min()

(10.0, 100.0)

In [18]:
temp.age.max(), temp.fare.max()

(50.0, 200.0)